In [1]:
include("JuliRay.jl")
Base.@irrational ° 0.0174532925199432957692369076848861271344 (big(pi)/big(180))

In [138]:
ε=10^(-9)

1.0000000000000005e-9

In [115]:
VERTEX=Int
EDGE=Array{VERTEX,1}
FACE=Array{EDGE,1}
CELL=Array{FACE,1}

Array{Array{Array{Int64,1},1},1}

In [184]:
function ℝ³⭢S³(p::Array{T,1}) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    end
    return [2p[1]/(1+p[1]^2+p[2]^2+p[3]^2),2p[2]/(1+p[1]^2+p[2]^2+p[3]^2),2p[3]/(1+p[1]^2+p[2]^2+p[3]^2),(-1+p[1]^2+p[2]^2+p[3]^2)/(1+p[1]^2+p[2]^2+p[3]^2)]
end

function ℝ³⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    elseif(θ<ε)
        return [p[1:3]...,0.0]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        P=p*cot((π/2-θ)/2+π/4)
        return ℝ³⭢S³(P)*r+c
    end
end

function S³⭢ℝ³(q::Array{T,1}) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(!(norm(q)≈1.0))
        print(norm(q))
        error("No Point on unit S³, $(norm(q))")
    end
    return [q[1]/(1-q[4]),q[2]/(1-q[4]),q[3]/(1-q[4])]
end

function S³⭢ℝ³(q::Array{T,1},θ::Real) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(θ<ε)
        return q[1:3]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        return S³⭢ℝ³((q-c)/r)*tan((π/2-θ)/2+π/4)
    end
end

function ℝ⁴⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(θ<ε)
        return [p[1:3]...,0.0]
    else
        R=1/sin(θ)
        O=[0,0,0,cot(θ)]
        return O+R*normalize(p-O)
    end
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector)
    e₃=NormalVector(p₁,p₂,p₃)
    e₁=OrthogonalVector(e₃)
    e₂=cross(e₃,e₁)
    fixedpopint=(p₁+p₂+p₃)/3
    A=hcat(e₁,e₂,e₃)*transpose(hcat(e₁,e₂,-e₃))
    b=fixedpopint-A*fixedpopint
    return A*q+b
end

function NormalVector(p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    A=hcat(p₁-p₄,p₂-p₄,p₃-p₄)
    return normalize([(-1)^i*det(A[deleteat!(collect(1:n),i),:]) for i ∈ 1:n])
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    e₃=NormalVector(p₁,p₂,p₃,p₄)
    e₁=OrthogonalVector(e₃)
    e₂=cross(e₃,e₁)
    fixedpopint=(p₁+p₂+p₃)/3
    A=hcat(e₁,e₂,e₃)*transpose(hcat(e₁,e₂,-e₃))
    b=fixedpopint-A*fixedpopint
    return A*q+b
end

Mirror (generic function with 2 methods)

In [475]:
p₁=[1,0,0,0]
p₂=[0,1,0,8]
p₃=[0,43,3,0]
p₄=[-3,0,99,1]

4-element Array{Int64,1}:
 -3
  0
 99
  1

4×3 Array{Int64,2}:
   4    3    3
   0    1   43
 -99  -99  -96
  -1    7   -1

4-element Array{Float64,1}:
 0.9916334266884489  
 0.020351500785555237
 0.038839630969857916
 0.12141024073786172 

In [478]:
dot(B,p₁-p₂),dot(B,p₃-p₂)

(0.0, 0.0)

In [439]:
OrthogonalVector([2,0,0,0])

4-element Array{Float64,1}:
 0.0
 1.0
 0.0
 0.0

In [197]:
function vertices(face::FACE)
    f=copy(face)
    v=union(vcat(f...))
    w=[v[1]]
    for j ∈ 1:(length(v)-1)
        i=findfirst(e->w[end] ∈ e, f)
        push!(w,filter(v->v≠w[end],f[i])[1])
        deleteat!(f,i)
    end
    return w
end

function PickFace(cell::CELL, v::RealVector)
    if(norm(v)≈0)
        error("vector v must be non-zero")
    end
    faces=vertices.(cell)
    fpts=(i->POINTS[i]).(faces)
    return cell[findmax([dot(v,+(pts...)) for pts ∈ fpts])[2]]
end

PickFace (generic function with 2 methods)

In [65]:
function SphericalPolygon(v::Array{T,1},θ::Real) where T<:RealVector
    if(θ<ε)
        return Polygon()
    n=length(v)
    v₀=ℝ⁴⭢S³(+(v...)/n,θ)
    v₁=v[1]
    v₂=v[2]
    v₃=v[3]
    V=(q->S³⭢ℝ³(q,θ)).(v)
    V₀=S³⭢ℝ³(v₀,θ)
    w=[ℝ⁴⭢S³((v[i]+v[mod(i,length(v))+1])/2,θ) for i ∈ 1:length(v)]
    W=(q->S³⭢ℝ³(q,θ)).(w)
    
    O=Circumcenter(V₀,V[1],V[2],V[3])
    sphere=Sphere(O,norm(V₀-O))
    N=NormalVector(V[1],W[1],V[2]);
    direction=sign(dot(N,O-V₀))
    
    cylinder=csgIntersection([
            (V₁=V[i];
            V₂=V[mod(i,n)+1];
            V₃=V[mod(i+1,n)+1];
            C=Circumcenter(V₁,W[i],V₂);
            N=NormalVector(V₁,W[i],V₂);
            direction=sign(dot(N,V₀-W[i]));
            cylinder=Cylinder(C,C+2*direction*norm(V₀-O)*N,norm(V₀-O)))
            for i ∈ 1:n
            ])
    return csgClip(sphere,cylinder)
end

SphericalPolygon (generic function with 1 method)

In [66]:
function SphericalCylinder(v₁,v₂,r::Real,θ::Real) where T<:RealVector
    if(θ<ε)
        return 
    end
    n=length(v)
    v₀=ℝ⁴⭢S³(+(v...)/n,θ)
    v₁=v[1]
    v₂=v[2]
    v₃=v[3]
    V=(q->S³⭢ℝ³(q,θ)).(v)
    V₀=S³⭢ℝ³(v₀,θ)
    w=[ℝ⁴⭢S³((v[i]+v[mod(i,length(v))+1])/2,θ) for i ∈ 1:length(v)]
    W=(q->S³⭢ℝ³(q,θ)).(w)
    
    O=Circumcenter(V₀,V[1],V[2],V[3])
    sphere=Sphere(O,norm(V₀-O))
    N=NormalVector(V[1],W[1],V[2]);
    direction=sign(dot(N,O-V₀))
    
    cylinder=csgIntersection([
            (V₁=V[i];
            V₂=V[mod(i,n)+1];
            V₃=V[mod(i+1,n)+1];
            C=Circumcenter(V₁,W[i],V₂);
            N=NormalVector(V₁,W[i],V₂);
            direction=sign(dot(N,O-V₀));
            cylinder=Cylinder(C,C-direction*norm(V₀-O)*N,norm(V₀-O)))
            for i ∈ 1:n
            ])
    return csgClip(sphere,cylinder)
end

SphericalCylinder (generic function with 1 method)

In [297]:
function NewCell(cell::CELL,v::RealVector)
    n=length(POINTS)

    c=copy(cell)
    face=PickFace(cell,v)
    IND_face=vertices(face)
    IND_cell=Int[]
    for f ∈ c for e ∈ f for v ∈ e push!(IND_cell,v) end end end
    IND_cell=union(IND_cell)
    println(IND_cell)
    println(IND_face)
#     IND_cell2=[if(i ∈ IND_face) i else push!(POINTS,POINTS[i]*2); n=n+1 end for i ∈ IND_cell]
    IND_cell2=[if(i ∈ IND_face) i else push!(POINTS,Mirror(POINTS[i],POINTS[IND_face[1]],POINTS[IND_face[2]],POINTS[IND_face[3]])); n=n+1 end for i ∈ IND_cell]
    println(IND_cell2)
    
    c2=[[[IND_cell2[findfirst(w->w==v,IND_cell)] for v ∈ e] for e ∈ f] for f ∈ c]
    
   return c2
end

NewCell (generic function with 1 method)

In [429]:
# Cube of Tesseract
h=1/2
r=√(1-h^2)
v₁, 𝒑₁=1, [1,1,1]/2
v₂, 𝒑₂=2, [1,1,-1]/2
v₃, 𝒑₃=3, [1,-1,1]/2
v₄, 𝒑₄=4, [1,-1,-1]/2
v₅, 𝒑₅=5, [-1,1,1]/2
v₆, 𝒑₆=6, [-1,1,-1]/2
v₇, 𝒑₇=7, [-1,-1,1]/2
v₈, 𝒑₈=8, [-1,-1,-1]/2
e₁=[v₁,v₂]
e₂=[v₁,v₃]
e₃=[v₁,v₅]
e₄=[v₂,v₄]
e₅=[v₂,v₆]
e₆=[v₃,v₄]
e₇=[v₃,v₇]
e₈=[v₄,v₈]
e₉=[v₅,v₆]
e₁₀=[v₅,v₇]
e₁₁=[v₆,v₈]
e₁₂=[v₇,v₈]
f₁=[e₁,e₂,e₄,e₆]
f₂=[e₁,e₃,e₅,e₉]
f₃=[e₂,e₃,e₇,e₁₀]
f₄=[e₄,e₅,e₈,e₁₁]
f₅=[e₆,e₇,e₈,e₁₂]
f₆=[e₉,e₁₀,e₁₁,e₁₂]
c₁=[f₁,f₂,f₃,f₄,f₅,f₆]
POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄,𝒑₅,𝒑₆,𝒑₇,𝒑₈]
POINTS=r*normalize.(POINTS)

8-element Array{Array{Float64,1},1}:
 [0.5, 0.5, 0.5]   
 [0.5, 0.5, -0.5]  
 [0.5, -0.5, 0.5]  
 [0.5, -0.5, -0.5] 
 [-0.5, 0.5, 0.5]  
 [-0.5, 0.5, -0.5] 
 [-0.5, -0.5, 0.5] 
 [-0.5, -0.5, -0.5]

In [430]:
c₁

6-element Array{Array{Array{Int64,1},1},1}:
 [[1, 2], [1, 3], [2, 4], [3, 4]]
 [[1, 2], [1, 5], [2, 6], [5, 6]]
 [[1, 3], [1, 5], [3, 7], [5, 7]]
 [[2, 4], [2, 6], [4, 8], [6, 8]]
 [[3, 4], [3, 7], [4, 8], [7, 8]]
 [[5, 6], [5, 7], [6, 8], [7, 8]]

In [431]:
c₂=NewCell(c₁,[1,2,0])

[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 6, 5]
[1, 2, 9, 10, 5, 6, 11, 12]


6-element Array{Array{Array{Int64,1},1},1}:
 [[1, 2], [1, 9], [2, 10], [9, 10]]    
 [[1, 2], [1, 5], [2, 6], [5, 6]]      
 [[1, 9], [1, 5], [9, 11], [5, 11]]    
 [[2, 10], [2, 6], [10, 12], [6, 12]]  
 [[9, 10], [9, 11], [10, 12], [11, 12]]
 [[5, 6], [5, 11], [6, 12], [11, 12]]  

In [432]:
c₃=NewCell(c₂,[1,2,0])

[1, 2, 9, 10, 5, 6, 11, 12]
[9, 10, 12, 11]
[13, 14, 9, 10, 15, 16, 11, 12]


6-element Array{Array{Array{Int64,1},1},1}:
 [[13, 14], [13, 9], [14, 10], [9, 10]]  
 [[13, 14], [13, 15], [14, 16], [15, 16]]
 [[13, 9], [13, 15], [9, 11], [15, 11]]  
 [[14, 10], [14, 16], [10, 12], [16, 12]]
 [[9, 10], [9, 11], [10, 12], [11, 12]]  
 [[15, 16], [15, 11], [16, 12], [11, 12]]

In [433]:
POINTS

16-element Array{Array{Float64,1},1}:
 [0.5, 0.5, 0.5]   
 [0.5, 0.5, -0.5]  
 [0.5, -0.5, 0.5]  
 [0.5, -0.5, -0.5] 
 [-0.5, 0.5, 0.5]  
 [-0.5, 0.5, -0.5] 
 [-0.5, -0.5, 0.5] 
 [-0.5, -0.5, -0.5]
 [0.5, 1.5, 0.5]   
 [0.5, 1.5, -0.5]  
 [-0.5, 1.5, 0.5]  
 [-0.5, 1.5, -0.5] 
 [0.5, 2.5, 0.5]   
 [0.5, 2.5, -0.5]  
 [-0.5, 2.5, 0.5]  
 [-0.5, 2.5, -0.5] 

In [434]:
θ=π/2
# R=1/sin(θ)
# O=[0,0,0,tan(θ)]
# N=O+[0,0,0,R]
# H=O-[0,0,0,√(R^2-r^2)]
# POINTS=[[P[1:3]...,0]+H for P ∈ POINTS]
POINTS=(p->ℝ³⭢S³(p,θ)).(POINTS)

16-element Array{Array{Float64,1},1}:
 [0.5714285714285715, 0.5714285714285715, 0.5714285714285715, -0.1428571428571426]   
 [0.5714285714285715, 0.5714285714285715, -0.5714285714285715, -0.1428571428571426]  
 [0.5714285714285715, -0.5714285714285715, 0.5714285714285715, -0.1428571428571426]  
 [0.5714285714285715, -0.5714285714285715, -0.5714285714285715, -0.1428571428571426] 
 [-0.5714285714285715, 0.5714285714285715, 0.5714285714285715, -0.1428571428571426]  
 [-0.5714285714285715, 0.5714285714285715, -0.5714285714285715, -0.1428571428571426] 
 [-0.5714285714285715, -0.5714285714285715, 0.5714285714285715, -0.1428571428571426] 
 [-0.5714285714285715, -0.5714285714285715, -0.5714285714285715, -0.1428571428571426]
 [0.2666666666666666, 0.7999999999999999, 0.2666666666666666, 0.4666666666666669]    
 [0.2666666666666666, 0.7999999999999999, -0.2666666666666666, 0.4666666666666669]   
 [-0.2666666666666666, 0.7999999999999999, 0.2666666666666666, 0.4666666666666669]   
 [-0.26666666666

In [435]:
function P2O(vertex::VERTEX,POINTS;rᵥ=0.05)
    return Sphere(POINTS[translate(vertex)],rᵥ)
end
function P2O(edge::EDGE,POINTS;rₑ=0.05)
    return Cylinder(POINTS[translate(edge)]...,rₑ)
end
function P2O(face::FACE,POINTS)
    v=vertices(face)
    return Polygon([POINTS[i] for i ∈ v])
end
function P2O(cell::CELL,POINTS;rᵥ=0.05,rₑ=0.05)
    fs=copy(cell)
    es=DeleteDuplicates(vcat([face for face ∈ fs]...))
    vs=DeleteDuplicates(vcat([edge for edge ∈ es]...))
    V=rgbColor(csgUnion([Sphere(S³⭢ℝ³(POINTS[vertex],θ),rᵥ) for vertex ∈ vs]),RGB(0.1,0.1,0.1))
    E=rgbColor(csgUnion([Arc(
                    S³⭢ℝ³(POINTS[edge[1]],θ),
                    S³⭢ℝ³(ℝ⁴⭢S³((POINTS[edge[1]]+POINTS[edge[2]])/2,θ),θ),
                    S³⭢ℝ³(POINTS[edge[2]],θ),
                    rₑ
                ) for edge ∈ es]),RGB(0.2,0.2,0.2))
    F=rgbftColor(SphericalPolygon([POINTS[i] for i ∈ vertices(fs[6])],θ),RGB(1,0,0),FT(0.1,0.1))
    return csgUnion(V,E,F)
end

P2O (generic function with 4 methods)

In [436]:
render(P2O(c₂,POINTS,rᵥ=0.05,rₑ=0.025),camera=LngLatCamera(lng=30°,lat=30°,pers=0.5,zoom=0.2,width=640,height=360),name="hgoe4")

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray hgoe4.pov`, ProcessExited(0))